# Setup

We will be using two repositories:


*   The actual liboqs C-based library: https://github.com/open-quantum-safe/liboqs
*   The Python wrapper for that library: https://github.com/open-quantum-safe/liboqs-python



A wrapper is a piece of code that provides an interface between two different systems or components, in this case liboqs (written in C) and Python.The liboqs-python wrapper is the code written in Python that "wraps around" the C library. Its purpose is to:

*   Translate: It translates calls from Python code into the format that the C library understands.
*   Simplify: It often simplifies the interface of the C library, making it easier to use from Python.
*   Handle Data Types: It manages the conversion of data types between Python and C.
*   Load the Library: It handles loading the shared C library into memory so that Python can access its functions.

In [1]:
# Install the necessary dependencies
!apt-get update
!apt-get install -y cmake build-essential git

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,305 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,561 kB]
Get:13 https://developer.download.nvidia.com/com

For the wrapper to work correctly, it is required to save the compiled files in the `/root/_oqs` directory, so set the parameter `-DCMAKE_INSTALL_PREFIX="/root/_oqs"`.

In [2]:
# Configure, build, and install liboqs, per https://github.com/open-quantum-safe/liboqs-python
!git clone https://github.com/open-quantum-safe/liboqs.git # creting liboqs directory in /content
!cmake -S liboqs -B liboqs/build -DCMAKE_INSTALL_PREFIX="/root/_oqs" -DBUILD_SHARED_LIBS=ON # save the compiled files in /root/_oqs
!cmake --build liboqs/build --parallel 8
!sudo cmake --build liboqs/build --target install

Cloning into 'liboqs'...
remote: Enumerating objects: 35523, done.
remote: Counting objects: 100% (404/404), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 35523 (delta 354), reused 298 (delta 288), pack-reused 35119 (from 4)
Receiving objects: 100% (35523/35523), 157.68 MiB | 19.10 MiB/s, done.
Resolving deltas: 100% (25637/25637), done.
-- The C compiler identification is GNU 11.4.0
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Performing Test CC_SUPPORTS_WA_NOEXECSTACK
-- Performing Test CC_SUPPORTS_WA_NOEXECSTACK - Success
-- Performing Test LD_SUPPORTS_WL_Z_NOEXECSTACK
-- Performing Test LD_SUPPORTS_WL_Z_NOEXECSTACK - Success
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Th

In [3]:
# Install the liboqs python wrapper
!git clone --depth=1 https://github.com/open-quantum-safe/liboqs-python # creting liboqs directory in /content
%cd liboqs-python
!pip install .

Cloning into 'liboqs-python'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 33 (delta 3), reused 16 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 26.08 KiB | 2.17 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/liboqs-python
Processing /content/liboqs-python
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for liboqs-python: filename=liboqs_python-0.12.0-py3-none-any.whl size=12437 sha256=2417e7d315c2b05e79d367af0717f97ea0b3aeb82f16df9837a0bb427a6e3092
  Stored in directory: /root/.cache/pip/wheels/88/6a/34/7b907e7c67601095491eb47ea991c49facb1ce071d5b40d09c
Successfully built liboqs-python


In [4]:
import oqs

/content/liboqs-python/oqs/__init__.py:1: UserWarning: liboqs version (major, minor) 0.13.1-dev differs from liboqs-python version 0.12.0
  from oqs.oqs import (


In [5]:
# Get the supported KEM algorithms
oqs.get_enabled_kem_mechanisms()

('BIKE-L1',
 'BIKE-L3',
 'BIKE-L5',
 'Classic-McEliece-348864',
 'Classic-McEliece-348864f',
 'Classic-McEliece-460896',
 'Classic-McEliece-460896f',
 'Classic-McEliece-6688128',
 'Classic-McEliece-6688128f',
 'Classic-McEliece-6960119',
 'Classic-McEliece-6960119f',
 'Classic-McEliece-8192128',
 'Classic-McEliece-8192128f',
 'Kyber512',
 'Kyber768',
 'Kyber1024',
 'ML-KEM-512',
 'ML-KEM-768',
 'ML-KEM-1024',
 'sntrup761',
 'FrodoKEM-640-AES',
 'FrodoKEM-640-SHAKE',
 'FrodoKEM-976-AES',
 'FrodoKEM-976-SHAKE',
 'FrodoKEM-1344-AES',
 'FrodoKEM-1344-SHAKE')

In [6]:
# Get the supported signature algorithms
oqs.get_enabled_sig_mechanisms()

('Dilithium2',
 'Dilithium3',
 'Dilithium5',
 'ML-DSA-44',
 'ML-DSA-65',
 'ML-DSA-87',
 'Falcon-512',
 'Falcon-1024',
 'Falcon-padded-512',
 'Falcon-padded-1024',
 'SPHINCS+-SHA2-128f-simple',
 'SPHINCS+-SHA2-128s-simple',
 'SPHINCS+-SHA2-192f-simple',
 'SPHINCS+-SHA2-192s-simple',
 'SPHINCS+-SHA2-256f-simple',
 'SPHINCS+-SHA2-256s-simple',
 'SPHINCS+-SHAKE-128f-simple',
 'SPHINCS+-SHAKE-128s-simple',
 'SPHINCS+-SHAKE-192f-simple',
 'SPHINCS+-SHAKE-192s-simple',
 'SPHINCS+-SHAKE-256f-simple',
 'SPHINCS+-SHAKE-256s-simple',
 'MAYO-1',
 'MAYO-2',
 'MAYO-3',
 'MAYO-5',
 'cross-rsdp-128-balanced',
 'cross-rsdp-128-fast',
 'cross-rsdp-128-small',
 'cross-rsdp-192-balanced',
 'cross-rsdp-192-fast',
 'cross-rsdp-192-small',
 'cross-rsdp-256-balanced',
 'cross-rsdp-256-fast',
 'cross-rsdp-256-small',
 'cross-rsdpg-128-balanced',
 'cross-rsdpg-128-fast',
 'cross-rsdpg-128-small',
 'cross-rsdpg-192-balanced',
 'cross-rsdpg-192-fast',
 'cross-rsdpg-192-small',
 'cross-rsdpg-256-balanced',
 'cross

# KEMs and Signatures in liboqs

## KEMs

In [39]:
# Alice creates an instance of the algorithm used as KEM
# This instance contains all the parameters and function pointers for the algorithm to function
alice_kem = oqs.KeyEncapsulation("FrodoKEM-640-AES")

# Generate the KEM keypair
alice_public_key_kem = alice_kem.generate_keypair()
print(f"Alice's KEM public key {alice_public_key_kem.hex()}")
print(f"Size: {len(alice_public_key_kem)} bytes")

print("--------------------")

# Retrieve the KEM private key
private_key_kem = alice_kem.export_secret_key()
print(f"Alice's KEM private key: {private_key_kem.hex()}")
print(f"Size: {len(private_key_kem)} bytes")

# It is possible to reconstruct a session using your private key when creating an instance
kem_later = oqs.KeyEncapsulation("FrodoKEM-640-AES", private_key_kem)

Alice's KEM public key 9f03fde56c67949be93593fc06ed852b64d8f906b21c04b8ef3669de34355f3f3b2fef5eef6709baab3292769cec865381f878392d1f9bf264363d9fb0717de7426e454ab824e9d5e16010f952043bb3ef34b1116cf9c722f2d8fa041fb21528eefe6515c37348c3efe2e298282a9d71ac1d0b3ced7af3fd8f36937bdd5bdbb552b67ba9456d6a8779c2df80856a63ea32c34f9b7e6a9f8a7ae31c4f1eef864b3eeb3ba3f23ad8ed5e2f3a47de8e86713c38c28170994753eb3420db9a4a6639c1d9f37d3449b1fe15f5313d86d7ecac8463a882cdd1298ac5177edb5b6c753e9c6fece5dba8b9a50c06301edb76ad795e0fc52999e89f502bc319019e73c0628e05065a1ec79f4f2b688efa249bfbda689566190e9a618256cf250bdba12b1f84afe6b33d1a5e2972f7795c010c2f009911d6930740a4a2d65c0e64c41c6e3c4c9794cc0c321095236bdbfa0d29c360c859b27190ae7187812628a948361dded5ada78005b9efc98bcdf6770e93af68fc4d6dd92099cd3840743d80c22baa25d22336e1192499bef59451ae64c65080ee1f987810daf6a40d4ee88444943e833891de991a94190e724c363766fdeb898585e40f2f6ad67890c4847e003cbcd4656f2274b412d556576aebf005e8de6a38134c0ed90c399c7295af826f0242fc2fff0a5babb161583

In [40]:
# Bob encapsulates the symmetric key using Alice's KEM public key
# The output of this function is a randomly generated symmetric key (symmetric_key_bob) and its ciphertext version (ciphertext)
bob_kem = oqs.KeyEncapsulation("FrodoKEM-640-AES")
ciphertext, symmetric_key_bob = bob_kem.encap_secret(alice_public_key_kem)

print(f"Symmetric key: {symmetric_key_bob.hex()}")
print(f"Size: {len(symmetric_key_bob)}")

print("--------------------")

print(f"ciphertext: {ciphertext.hex()}")
print(f"Size: {len(ciphertext)}")

Symmetric key: 089c0ea61eaa77b4fa1a34446f4d998f
Size: 16
--------------------
ciphertext: b948dc0bad5ac0e9fb4251ff4105575c6026d78bc35387364150014c7a1a5a36274b08c066996cd49e4df954d302453a4b585642ae2c0129648d0147437ba69315a8a01303e9bd9ff5feb66348e8aba67a05834bf9375c796a39e204f7079f7142225071fb543866d3f94e10dee32955c2c377b5ea1635031168f84ecdec8ccaecccaa0da4ed6a67796c5f37a8d3a0ed3eb70fec7d2774c169aec5c3067e002ce659ecb9ef4354ee933061372bdab7648f01b68ec92c19327f99b1109877d40bc3fe68e8eabdb3952db7ee6c53ba213b7d289c58ea0404005b251129c2c818a89356b4b8772a033bf789fc2c86c72404fafe1cf429f07671a74e1cccda9871121923b896d9b58deec5fce642d2242556860793491837d6d1d1088bd2acc2200135def6457f10625495aeb65459f71facea839e40b0409b6eb810bbcf7c9468ef90bb0a15e1529b4cdf680534b2beaf053395f9dd1b67e573dd0725dea9c5bfe932807cb6201192b1f199fd2ef2926ff0f6287521c6d53c0901f5db6475c11ffc136a2186c3dea699e84007cccb696b540c605de0c06c69dc71918756025cba37665f6b080b16fd8666d40cfe1b940d443546263cfd04fefd50ba57bf3762d15433f8aacbe04f7a

In [41]:
# Alice decapsulates the ciphertext to obtain the shared secret
symmetric_key_alice = alice_kem.decap_secret(ciphertext)

print(f"Symmetric key: {symmetric_key_alice.hex()}")
print(f"Size: {len(symmetric_key_alice)}")

Symmetric key: 089c0ea61eaa77b4fa1a34446f4d998f
Size: 16


In [42]:
# Verify that the secret keys match
if symmetric_key_alice == symmetric_key_bob:
  print("Symmetric keys match! Use it for symmetric encryption.")
else:
  print("Symmetric keys did not match...try again.")

Symmetric keys match! Use it for symmetric encryption.


Your are going to creaagree on a shared secret with a colleague to exchange encrypted messages using the quantum resitant algorithm BIKE-L1, which is available in liboqs. To do so, follow the next steps:

- Person A will generate a public key and post it in an entry [here](https://padlet.com/pablogf/hands-on-pqc-at-opensouthcode-4957mbhgd0u8486m).
- Person B will encapsulate a key for Person A using their public key.
  - *Tip: All functions work with bytes. When copying the public key from the dashboard into the colab, make sure to use `bytes.fromhex()`.*
- Person A will decapsulate it, and use it to encrypt a message of your choice using AES-256 CBC mode in [cryptii.com](https://cryptii.com/) (make sure to leave the IV section as it is).
- Person A will post the encrypted message in the comments of that same entry.
- Person B will have to decrypt it using the same tool and configuration. Post the resulting plaintext in the dashbaord to verify it is correct!

Now you have a key to communicate with!

Use the cell below to perform the role of Alice:

In [17]:
# Create an instance of the algorithm used as KEM

# Generate a KEM keypair


In [18]:
# Decapsulate


Use the cell below to perform the role of Bob:



In [71]:
# Encapsulate


## Digital Signatures

In [33]:
# Alice creates an instance of the algorithm used for signing
alice_sig = oqs.Signature("SPHINCS+-SHA2-128f-simple")

# Alice generates a signature keypair
alice_public_key_sig = alice_sig.generate_keypair()
print(f"Signature public key: {alice_public_key_sig.hex()}")
print(f"Size: {len(alice_public_key_sig)} bytes")

print("--------------------")

# Alice retrives the signature private key
private_key_sig = alice_sig.export_secret_key()
print(f"Signature private key: {private_key_sig.hex()}")
print(f"Size: {len(private_key_sig)} bytes")

Signature public key: 27aa58e0fb1164b0c8d28a713d54506bccee1319c976dc2490e5f882bf3138a4
Size: 32 bytes
--------------------
Signature private key: da8d506885162806e99e6e5d67122716428c2ef74f117f443fb5158b48abc59227aa58e0fb1164b0c8d28a713d54506bccee1319c976dc2490e5f882bf3138a4
Size: 64 bytes


In [34]:
# Sign the message
input_string = "test"
signature = alice_sig.sign(input_string.encode("utf-8"))
print(f"Signature: {signature.hex()}")
print(f"Size: {len(signature)} bytes")

Signature: ...
Size: 17088 bytes


In [35]:
# Bob creates an instance of the algorithm the message was signed with
bob_sig = oqs.Signature("SPHINCS+-SHA2-128f-simple")

# Bob verifies the signature
is_valid = bob_sig.verify(input_string.encode("utf-8"), signature, alice_public_key_sig)
print(f"Is the signature valid? {is_valid}")

Is the signature valid? True


### Your turn!

You are going to create a quantum-resitant signature using the soon-to-be-standardized Falcon-1024 algorithm. Follow the next steps:

- Person A will generate a public key, come up with any message they want, and sign it.
- Person A will then create an entry [here](https://padlet.com/pablogf/hands-on-pqc-at-opensouthcode-4957mbhgd0u8486m) containing the signature, the message being signed, and their public key.
  - Besides this, make sure to post several different tweaked versions of the signature by altering a character of the signature. This way, the verifier will have to find which of the signatures is the correct one.
- Person B will verify the signatures posted, and will respond in a comment to the entry stating which one is the valid signature.
  - *Tip: Remember that all variables must be worked with in bytes. Use `bytes.fromhex()` when pasting the public key and the signature from the dashboard into the notebook. For the input string make sure to use `.decode("utf-8").`*

Use the cell below to perform the role of Alice:

In [36]:
# Create an instance of the algorithm used for signing

# Generate a signature keypair

# Sign the message


Use the cell below to perform the role of Bob:

In [37]:
# Verify the signature


# PQC protocol using ML-KEM and ML-DSA

You are now ready to program an authenticated quantum-secure communication protocol using liboqs! Let's do it using the algorithms standardized by NIST: ML-KEM-512 as the key encapsulation mechanism and ML-DSA-44 as digital signature.

## On Alice's end...

Alice generates keys for KEM:

In [43]:
# Create an instance of the algorithm used as KEM
# This isntance contains all the parameters and function pointers for the algorithm to function
alice_kem = oqs.KeyEncapsulation("ML-KEM-512")

# Alice generates the KEM keypair
alice_kem_PK = alice_kem.generate_keypair()
print(f"Alice's KEM public key {alice_kem_PK.hex()}")
print(f"Size: {len(alice_kem_PK)} bytes")

print("--------------------")

# Retrieve the KEM private key
alice_kem_pk = alice_kem.export_secret_key()
print(f"Alice's KEM private key: {alice_kem_pk.hex()}")
print(f"Size: {len(alice_kem_pk)} bytes")

Alice's KEM public key a82c977613ae1d2157f295680a1aad20db1eeed69cbf197b5471590f3c3f40525a10f18d88827a75d3c89db518e810590e0b5264445e733c1342f7b88788af918196d4cb129a44b539f1cf6660c41584b91981c3c6b514fa23c61229873f192a101b2ff068503c6770e61b7f3614b16484b1518919de36288abbc3007480824b67d21798c4e828af7355c676af39e22f0b385e847c02d7b477d71bcf6cdb97434c7d4b6b5126d5700fa02b51d519c1c907f18a254a2260e4999b627324d57676267a678e9767e79304c91a01daa79f0107396c0a19ddbc0a16c230c96477bdc9cd90db2ea2eab35536c311940e6b0a9dd4d650a7c076cc4a7c6df98df920c63113708fc1b8262c706f7c4654f026a2b4b199e5186a627ca0c76a8d31828be526709152e821c6cbb4954b4a73a3d5c574f01b7f9021f53b9958207bd3f5ad564b94a6693db1d5b9903939337317364799496862a39a4311a0bf94c279f58cc0e2015bdd355876973512717e6c154c610b6ddc795767f67a51663399f9886af298aa727c7935caeea261ed6100f7b94880840005dcb664589d85c65395a62f3c068058b84b68540f7e71c1f662c40f01592842b40fe2a3867098116b7fb9f8b6ffe33de153269ab662b7b2ca4c8015cb0a222ad134de86703cb08328644b6529b5ff8a7abe1602d5c72

Alice generates keys for authentication:

In [44]:
# Create an instance of the algorithm used for signing
alice_sig = oqs.Signature("ML-DSA-44")

# Alice generates a signature keypair
alice_sig_PK = alice_sig.generate_keypair()
print(f"Alice's signature public key: {alice_sig_PK.hex()}")
print(f"Size: {len(alice_sig_PK)} bytes")

print("--------------------")

# Retrive the signature private key
alice_sig_pk = alice_sig.export_secret_key()
print(f"Alice's signature private key: {alice_sig_pk.hex()}")
print(f"Size: {len(alice_sig_pk)} bytes")

Alice's signature public key: 008f47f6414ed7ec235406daf3e469cb6c0eba33469205259d3859e50c648c526113094cebebbc52d93a9324beed0dce7e6087af683b429debbc631e6d0658fa34d6ae249b61a77351f964307aab96a0a6199b181657781b628c2c748814469b4a811b4dedbf7cd81aacf5e081f5ce38e3e7485661c0e4a5f2e17bb2ac1c9700f2273a234085690b5a9325ee66e501f410d66794d5bf685fa4a8610f07526f908935b76dee7964c39907304047b3e8261d288f1578bd523ea9645d8ba19a9ebe7812e3460df0034347b2fd6562cc4ff37adc8a5a152c75cd409db2a03cb19b056626809cae7f9705b973d3965a9dc8b59803013bf0769c0eb21a6638928b5cd1cb16abdfbf5ebe6ffbd598b615b71b7275437ec7c9e08e4aa4fcd218cc5304ee0b5c2d5e712dbe588b9b091ff2f5c57711fd46e04c35a20f3cca90fc67374cd8f0a09219de2fde949d253ba31950d773421a91e12b64beec22530182a8529e960ebcd5319653cc2ce5dfe88f6752dd40ceb984b1a3a9564241afa88198ef63ff79e31fffdb856a92d6c404742eaabdfb9920b6d0c17d600e38a2d4af62ae5f53a9feafd25b89bdfd9249bce876a643a3d094d1b0cccc02792a7bb66595105ad1ff106a48232b2a39630a453c1c0e59866aee0364e94dc418c0a10226a0501b3f7a456bcf26

Alice signs her public key to authenticate it:

In [45]:
# Alice signs her KEM public key for authentication and sends it to Bob
signature_alice = alice_sig.sign(alice_kem_PK)
print(f"Alice's signature of her public key: {signature_alice.hex()}")
print(f"Size: {len(signature_alice)}")

Alice's signature of her public key: 112bbc17595a1557c6fc5d1585232c79258b8c5121a51fb2c0e0a5b6ac6a011d7761902abce3f33c09ad47b38516fa4611f95dd9513b62015a064ebaaa4b0d065ab20aed0c643a503d86e9739e8713d16b6af591773779013b6f928c3a701411802b3e4f5a0da8cef3a0fdce9c73ec928a7764d01c255914f8f0223406a6eb91ec27fc81efa38aef96c812cb17fd9376f6622bf6a87bf377c153d4cc600441597696b2bf31b901c647c266bdefbffa6f014b63e8f2c24609913cf47c89ac242d375a096bea3498ee807361dbaf56d41de3de1ae429fbe0b240b957f795fde8ed33dba0535f63364439ad4cf6798c325df2763da22d109cccec1e4f204de640b8a4cb125f349ba4fb6817dfde23ac37dd92df3286a02341b54330191f5d2ba77d5ce49c7b607d786e6bbde31bd4f449a387563b639bda259ff040c9ef2dc8cf4a27179ea2ee40ee64a92188c861d73be7b193a67995e46e36d8d93592df2ca7d1d99c961930a2c6b89597baa2c737a931795f47c1482585d4427faf0eca2307ad69aefc54532aa7ed4dc867bb0d9e509f578a2b9c4fdee9e7bf690283240292b056a7ee8796e9177cacdbff2f2946bf6927418c640e51693cb23454d24a0ca03100ae9dd8ca01f827a981c1241856d26ed608973429b0e9774ec47427b9d1a020f82

## On Bob's end...

Bob verifies Alice's signature:

In [46]:
# Create an instance of the algorithm that was used for the signature
bob_sig = oqs.Signature("ML-DSA-44")

# Bob verifies Alice's signature
is_alice_real = bob_sig.verify(alice_kem_PK, signature_alice, alice_sig_PK)
print(f"Is Alice who she claims to be? {is_alice_real}")

Is Alice who she claims to be? True


Then, he moves on to encapsulate a symmetric key:

In [47]:
# Bob encapsulates the symmetric key using Alice's KEM public key
# The output of this function is: a randomly generated symmetric key (symmetric_key) and its ciphertext version (ciphertext)
bob_kem = oqs.KeyEncapsulation("ML-KEM-512")
ciphertext, symmetric_key_bob = bob_kem.encap_secret(alice_kem_PK)

print(f"Symmetric key: {symmetric_key_bob.hex()}")
print(f"Size: {len(symmetric_key_bob)}")

print("--------------------")

print(f"ciphertext: {ciphertext.hex()}")
print(f"Size: {len(ciphertext)}")

Symmetric key: b86a0f193ef1a2e11f573369db75dc77c6eabe8846d85cd4ec06ce1c1cd803c4
Size: 32
--------------------
ciphertext: 290cb530e68c016118896efd3f58a0fb83e3786e95e347b5c985a55cc22cd181a2f2c58a49181a3d59be31be2e62c691f78edfec872560243e6624e595288116d0055b1c97831d8cdda48915a8932b5833a39316e15b7d29dc265af1e04ca467d55664256ffd7326821fc5af975c7b13b18402529a48541513af4e634fa5403f6721408206f1eff08856a1070c82f6767e741dd6e6bbf2aa1fb2c7686c7afa9d2a6047b30d8c2fe0216bd422a07b4734349ac6baa91f61247d5d886e3a8817daa951868c17ce8d5d9d9c5f178ea53fca879f8929df792c418cf2128344e8d1492e7bea68cf5b5a36a4817099aacf2d1a19ce50e51ea77b2395820d7d1d8268fe0b7eedb52dc88e1dbebe0de06a7cab91d2ea7d5945ef56c9f88e7c8a701d6353a36744197f0d6614fad4cd0b23e96ff8369c1dea3101e93c716641331b2cda8f3672a859c2ecbd7fe835df355c98f2c10de4fea1bff4f735e7bd64f1159780ab62fa2d54d2a5cc5be249549509e8f3062b6691bca9cac28eafed2c69af36dc7cf04bad7a1700e5d807e754181a26c1c0c07f541e7d9c6a0a1bd22356eea0bf9990a9125c2d3fea5db0045dae6d7cf1cd5a767870d62d98

Now he creates a signature of the ciphertext to prove authenticity of the message sent:

In [48]:
# Create an instance of the algorithm used for signing
#bob_sig = oqs.Signature("ML-DSA-44")

# Bob generates a signature keypair
bob_sig_PK = bob_sig.generate_keypair()
print(f"Bob's signature public key: {bob_sig_PK.hex()}")
print(f"Size: {len(bob_sig_PK)} bytes")

print("--------------------")

# Retrive the signature private key
bob_sig_pk = bob_sig.export_secret_key()
print(f"Bob's signature private key: {bob_sig_pk.hex()}")
print(f"Size: {len(bob_sig_pk)} bytes")

Bob's signature public key: 38a85badd9d8de447d0b712f37eaacdec63bb8fa013abc721a7f88c82a779e76b23f9eeebe9b59f8a2af322136dbef7f877da05f1e7d9831b6b49fd4e697d07b4b42908680171059cd2d5a837f64e22b44add1eff24a8227e147aa9813dbaf31be166ea1a953c27725d98b6e4e880b1a0165b6231852f7e6b747413ee9f8d4f8dd213d25359557c8dbbb9a141e83c242c97d32d4187964bd659d8f034e3c85f125c0fe271547e05b5c548598201ac9e55f0a48defd040f672c69749000b5ce33bac1658bda5d4551e66c9c03b4b7c86a5b883a7306fd1860fb0af8765ecf988dc852624abb4134ea4381bfbfe74d6a526279522a268c542e184ad16d117d82d3a1fb2475bb2209eef772169a5224ee27e58f0b23dfbc0e8af5c5d923570c0aa13ee7df0449de9b16fa98b7336c061d2bb7256491a7d7cf40e618a0e3bf767cf7536aa913aa72a3c5d0d9d05891fa854ac2933f0ae81749c02504523ec34c07b353bffd56eec4ba68456aa6a842be2b875241f1960c0c3e9e70a0eda676cc1ec8443c050f245431b9eec86fb969cc7f6bc8857267c4e7ae7f3d741d483004044449f69e4ae5fba6d8eecab482e6d76538c5e173b43a96503a5ddd394f23fef146c27fe25a8d23891ac89d87904fa5ef3fb766fb1f097018e08604f9ec3d84db2ced8a71a50461

In [49]:
# Bob signs the ciphertext for authentication and sends it to Alice
signature_bob = bob_sig.sign(ciphertext)
print(f"Bob's signature of the ciphertext: {signature_bob.hex()}")
print(f"Size: {len(signature_bob)}")

Bob's signature of the ciphertext: e3137d5eac2bc516f04587d1daf127f6e961af996a659925301424c04d39b354825dc8330792445d1949e4c2c41a056e0ea13c12d0e7906ebd053ae6fcb090c15ceff4859ba262b593ddc4a5c2d9eb2ed564519332cd4bf8abfa357a9e0213f06528a7abc575729b0b0faea2bd70d41fe3947852adcd7640d805421309f98f634010240f2ef3e11f962829dc68c6d3f3247f01fe04cec74173489708e81a6f44a2740673b38890fc89cda4b9c345eef09f72b7837f3782486ddb988ff2d3d054e607b50edd5727662e55fcb2417c6b7db1d99addbddf75d9932efa392f03a05cb0b0aa8d2deef647b30d93903ab00388a5bbfbad043894a9f7df0da52e5d01073c1c81f220a09b4961109ae60a4af2485ce5883a710726197fde2c9e6796579b60824e7ada0c535f08e3bd237c74db10fc9e61cae859d5ec730dcd3018adfe1ffbf974f1963a9a26b9f99a524228edec5ee9782a25740bbc24750e2491d004e67df0128f60d36d9c84ce6a1d5b657d05f162a8fe5f686fd0a43f25d0476ab1edf0d23d42b6751a087de8e0c499a8d69ae688b8e24fdd708f8f9ec4899a148e0092cabf2fcb517ce8a2af86ad8f40a08253322a9710210c6d8ae91af3fa00e59417cc2696c02e5c1746ab4327024adf210f40e095173eb2cc1c995e8908eae35f34ded

## Back to Alice...

Alice verifies the authenticity of Bob's message:

In [50]:
is_bob_real = alice_sig.verify(ciphertext, signature_bob, bob_sig_PK)
print(f"Is Bob who he claims to be? {is_bob_real}")

Is Bob who he claims to be? True


Then she decapsulates the symmetric key:

In [51]:
symmetric_key_alice = alice_kem.decap_secret(ciphertext)

print(f"Symmetric key: {symmetric_key_alice.hex()}")
print(f"Size: {len(symmetric_key_alice)}")

Symmetric key: b86a0f193ef1a2e11f573369db75dc77c6eabe8846d85cd4ec06ce1c1cd803c4
Size: 32


In [52]:
# Verify that the secret keys match
if symmetric_key_alice == symmetric_key_bob:
  print("Symmetric keys match! Use it for symmetric encryption.")
else:
  print("Symmetric keys did not match...try again.")

Symmetric keys match! Use it for symmetric encryption.


Now Alice can use that symmetric key with a symmetric cipher like AES to start to securely communicate with Bob:

In [53]:
# Install pycryptodome for symmetric encryption with AES
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.2 MB/s eta 0:00:00


In [61]:
# Retrieve message from the user

from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Random import get_random_bytes

message = input("Type a message: ")

# Generate a random initialization vector (IV)
iv = get_random_bytes(16)

# Create the cipher object
cipher = AES.new(symmetric_key_alice, AES.MODE_CBC, iv)

# Pad the message to AES block size (16 bytes) and encrypt
padded_data = pad(message.encode('utf-8'), AES.block_size)
ciphertext = cipher.encrypt(padded_data)

# Combine IV and ciphertext (IV is needed for decryption)
encrypted_data = iv + ciphertext

print("Message to encrypt: ", encrypted_data.hex())

Type a message: hola
Message to encrypt:  8d84751e2e73ee22692e7687ffbf75940d40d5f357839eeb2cf32a58ee46eb27


## Back to Bob...

Bob decrypts the cipehertext from Alice with the shared secret acquired during the key encapsualtion process:

In [65]:
from Crypto.Util.Padding import unpad

ciphertext_hex = input("Ciphertext to decrypt: ")

# Change ciphertext from base64 to bytes
encrypted_data = bytes.fromhex(ciphertext_hex)

# Split IV and ciphertext
iv = encrypted_data[:16]
ciphertext = encrypted_data[16:]

# Decrypt
cipher = AES.new(symmetric_key_alice, AES.MODE_CBC, iv)
decrypted_padded = cipher.decrypt(ciphertext)

# Remove padding
plaintext = unpad(decrypted_padded, AES.block_size)

print("Decrypted message:", plaintext.decode('utf-8'))

Ciphertext to decrypt: 8d84751e2e73ee22692e7687ffbf75940d40d5f357839eeb2cf32a58ee46eb27
Decrypted message: hola
